In [1]:
# Install detectron2 -> Uncomment when connecting to Colab Kernel
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [2]:
import torch, detectron2
#!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

torch:  2.9 ; cuda:  cu126
detectron2: 0.6


In [1]:
import os
import math
import torch

from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.data import build_detection_train_loader
from detectron2.utils.logger import setup_logger

In [2]:
from utils.registerCable import register_cable
from utils.customMapper import CableDatasetMapper
from model.meta_arch import CableRCNN

register_cable()

In [3]:
# Add data augmentation
from detectron2.data import transforms as T
# Define a sequence of augmentations:
augs = [
    T.RandomBrightness(0.9, 1.1),
    T.RandomFlip(prob=0.5),
    T.RandomCrop("absolute", (700, 700)),
    T.RandomRotation(angle=[-10, 10]),
    T.RandomBrightness(0.8, 1.2),
    T.RandomContrast(0.8, 1.2),
    T.RandomSaturation(0.8, 1.2),
]

In [4]:
class MyTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        mapper = CableDatasetMapper(
            cfg,
            is_train=True,
            augmentations=augs,  # your existing augmentations
        )
        return build_detection_train_loader(cfg, mapper=mapper)


In [5]:
from detectron2.config import get_cfg

def setup_cfg(config_file, overrides=None):
    cfg = get_cfg()

    cfg.MODEL.LINE_LOSS_WEIGHT = 0.0

    cfg.merge_from_file(config_file)

    if overrides:
        cfg.merge_from_list(overrides)

    cfg.freeze()
    return cfg

In [6]:
cfg = setup_cfg(
    "model/maskrcnn_cable.yaml",
    overrides=[
        "MODEL.LINE_LOSS_WEIGHT", "0.0",
        "SOLVER.MAX_ITER", "40000",
        "OUTPUT_DIR", "output/cable",
    ],
)
setup_logger()
print(cfg.OUTPUT_DIR)
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

Config 'model/maskrcnn_cable.yaml' has no VERSION. Assuming it to be compatible with latest v2.


output/cable


In [7]:
trainer = MyTrainer(cfg)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
trainer.resume_or_load(resume=False)
trainer.train()